In [33]:
from heapq import heappop, heappush

# Manhattan distance (heuristic function)
def manhattan_distance(pointA, pointB):
    return abs(pointA[0] - pointB[0]) + abs(pointA[1] - pointB[1])

# A* algorithm with obstacle handling
def a_star(start, goal, grid):
    """
    A* algorithm to find the shortest path in a 2D grid with obstacles.
    
    Parameters:
    start (tuple): Start point (x, y)
    goal (tuple): Goal point (x, y)
    grid (list of lists): 2D grid where 0 is an empty space and 1 is an obstacle
    
    Returns:
    list: The shortest path from start to goal (as a list of points), or None if no path exists.
    """
    # Directions for moving in 4 possible ways (up, down, left, right)
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    rows = len(grid)
    cols = len(grid[0])
    
    # Priority queue for open set (nodes to be evaluated), stores (f_cost, point)
    open_set = []
    heappush(open_set, (0, start))
    
    # Dictionary to store the cost of the best path to each point
    g_score = {start: 0}
    
    # Dictionary to reconstruct the path
    came_from = {}
    
    # Set to store visited nodes
    visited = set()
    
    while open_set:
        # Get the point with the lowest f_cost
        current_f_cost, current = heappop(open_set)
        
        # If we reached the goal, reconstruct the path
        if current == goal:
            return reconstruct_path(came_from, current)
        
        visited.add(current)
        
        # Explore neighbors (up, down, left, right)
        for direction in directions:
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            
            # Check if the neighbor is within the bounds and not an obstacle
            if (0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and
                grid[neighbor[0]][neighbor[1]] == 0 and neighbor not in visited):
                
                # Tentative g_score (distance from start to the neighbor)
                tentative_g_score = g_score[current] + 1  # Assume all moves cost 1
                
                # If the neighbor has not been visited or we found a shorter path to it
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_cost = tentative_g_score + manhattan_distance(neighbor, goal)
                    heappush(open_set, (f_cost, neighbor))
    
    # If no path is found
    return None

def reconstruct_path(came_from, current):
    """
    Reconstruct the path from start to goal.
    
    Parameters:
    came_from (dict): A dictionary with the backtrace from each point to the previous point.
    current (tuple): The current point (goal) to start backtracking from.
    
    Returns:
    list: The shortest path as a list of points.
    """
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]  # Reverse the path to get it from start to goal

# Example grid (0 is an empty space, 1 is an obstacle)
grid = [
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 1],
    [1, 1, 0, 0, 0],
    [0, 1, 1, 1, 0]
]

start = (0, 0)
goal = (3, 4)

# Find the path
path = a_star(start, goal, grid)

if path:
    print("Shortest path found:", path)
else:
    print("No path found.")


No path found.


In [1]:
import numpy as np

In [2]:
array = np.zeros(shape=(20, 20))

In [22]:
def find_probable_points(current_state: tuple[int, int], layout: np.ndarray) -> list[tuple[int, int]]:
    """Algorithm for extract all possible ways to go from the current state.
    
    Parameters
    ----------
    current_state : State
        Current cell where ghost resides.
    layout : np.ndarray
        Layout of the maze.
    
    Returns
    -------
    list[State]
        List of states where a Ghost can go.
    """
    min_x, max_x = 0, layout.shape[1] - 1
    min_y, max_y = 0, layout.shape[0] - 1
    probable_states = []
    
    if current_state[0] > min_x:
        probable_states.append((current_state[0] - 1, current_state[1]))
    if current_state[0] < max_x:
        probable_states.append((current_state[0] + 1, current_state[1]))
    if current_state[1] > min_y:
        probable_states.append((current_state[0], current_state[1] - 1))
    if current_state[1] < max_y:
        probable_states.append((current_state[0], current_state[1] + 1))
    
    return probable_states

def _find_min_manhattan_distance(
    point_a: tuple[int, int], 
    point_b: tuple[int, int],
    layout: np.ndarray,
    manhattan_matrix: np.ndarray
) -> int:
    if point_b == point_a:
        manhattan_matrix[point_b[1], point_b[0]] = 1
        return 1
    
    if manhattan_matrix[point_a[1], point_a[0]] > 0:
        return manhattan_matrix[point_a[1], point_a[0]]
    
    manhattan_matrix[point_a[1], point_a[0]] = min(
        [
            _find_min_manhattan_distance(point_a=probable_point, point_b=point_b, layout=layout, manhattan_matrix=manhattan_matrix)
            for probable_point in find_probable_points(layout=layout, current_state=point_b)
        ]
    )
    return manhattan_matrix[point_a[1], point_a[0]]

In [23]:
manhattan_matrix = np.zeros_like(array)

_find_min_manhattan_distance(
    point_a=[0, 0],
    point_b=[19, 19],
    layout=array,
    manhattan_matrix=manhattan_matrix
)

RecursionError: maximum recursion depth exceeded in comparison

In [24]:
manhattan_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.